<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/L09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum Software Development
# Lab 9: Shor's Factorization Algorithm
# Copyright 2024 The MITRE Corporation. All Rights Reserved.

# Note: Use little endian ordering when storing and retrieving integers from
# qubit registers in this lab.

In [11]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit.library import MCXGate, UnitaryGate
from qiskit.visualization import plot_histogram
from math import ceil, log2, pow
import numpy as np

In [18]:
# E01

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    # Shift and multiply by constant
    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    # Swap to target register
    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    # Inverse shift and multiply by inverse constant
    inv_c = pow(c, -1, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

# Quantum modular exponentiation
def exp_mod(circ, a, b, input_qubits, output_qubits):
    # Placeholder for measurement and classical exponentiation
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    # Function to convert classical bit string to integer
    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    # Create a classical computation placeholder
    circ.append(UnitaryGate(np.eye(2**len(output_qubits))), output_qubits)  # Identity operation as a placeholder

    return classical_exp_mod

# Example usage in a circuit
number_to_factor = 15
guess = 7
input_size = ceil(log2(number_to_factor + 1))
output_size = input_size * 2

input_qubits = QuantumRegister(input_size, 'input')
output_qubits = QuantumRegister(output_size, 'output')
classical_bits = ClassicalRegister(input_size + output_size, 'c')

qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

# Apply Hadamard gates to input qubits
qc.h(input_qubits)

# Perform modular exponentiation
exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

# Measure the qubits
qc.measure(input_qubits, classical_bits[:input_size])
qc.measure(output_qubits, classical_bits[input_size:])

# Simulate the circuit
sim = AerSimulator()
t_qc = transpile(qc, sim)
qobj = assemble(t_qc)
result = sim.run(qobj).result()
counts = result.get_counts(qc)

counts = result.get_counts()
print(counts)

{'0110 000000000110': 68, '0001 000000000001': 65, '0101 000000000101': 64, '1100 000000001100': 67, '1011 000000001011': 53, '1000 000000001000': 66, '1101 000000001101': 60, '1110 000000001110': 66, '0000 000000000000': 67, '1111 000000001111': 62, '0011 000000000011': 66, '0100 000000000100': 60, '0010 000000000010': 75, '1001 000000001001': 64, '0111 000000000111': 58, '1010 000000001010': 63}


<ipython-input-18-2097764a1855>:70: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = sim.run(qobj).result()


In [ ]:
# E02

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply the exp_mod circuit
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Measure the qubits
    qc.measure(output_qubits, classical_bits)

    return qc

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts(qc)

    # Execute the circuit
    # backend = Aer.get_backend('qasm_simulator')
    # tqc = transpile(qc, backend)
    # qobj = assemble(tqc)
    result = execute(qc, backend).result()

    #counts = result.get_counts()
    print(counts)

In [ ]:
# E02

def inverse_qft(qc, qubits):
    qc.append(QFT(len(qubits), inverse=True).to_instruction(), qubits)

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Apply the inverse QFT to input qubits
    inverse_qft(qc, input_qubits)

    # Measure the input qubits
    qc.measure(input_qubits, classical_bits)

    return qc

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts(qc)

    # Execute the circuit
    # backend = Aer.get_backend('qasm_simulator')
    # tqc = transpile(qc, backend)
    # qobj = assemble(tqc)
    result = execute(qc, backend).result()

    #counts = result.get_counts()
    print(counts)

In [ ]:
# E03

from sympy import Rational

def find_period_candidate(numerator, denominator, denominator_threshold):
    fraction = Rational(numerator, denominator)
    continued_fraction = fraction.continued_fraction()
    convergents = continued_fraction.convergents()

    for convergent in convergents:
        if convergent.q < denominator_threshold:
            candidate = convergent
        else:
            break
    return (candidate.p, candidate.q)

In [ ]:
# E04

def find_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply the exp_mod circuit
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Measure the qubits
    qc.measure(output_qubits, classical_bits)

    return qc

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts(qc)

    # Execute the circuit
    # backend = Aer.get_backend('qasm_simulator')
    # tqc = transpile(qc, backend)
    # qobj = assemble(tqc)
    result = execute(qc, backend).result()

    #counts = result.get_counts()
    print(counts)